In [1]:
import cv2
import numpy as np
from numpy import asarray
import pandas as pd
from PIL import Image
import os
import math
import time
import tensorflow as tf



In [2]:
model = tf.keras.models.load_model('/Users/aly/Documents/Programming/Apps/Machine Learning/ASL Converter/MS-ASL/two_var/two_var_models/ResNet_four_var_weights.25-0.62')

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-15 22:28:09.705261: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-15 22:28:09.705361: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
saved_classes = ["milk", "coffee", "door", "dog"]

In [4]:
#changing the frame into a file
def get_files(frames):
    mode=0o666
    for fraaame in frames:
            print(fraaame)
    try:
        #if folder exists, overwrite it   
        # if os.path.isdir("training_models/demo_videos") == False:
        #     os.mkdir("training_models/demo_videos", mode)

        #saves files
        np.save("training_models/demo_videos/demo_video_file", np.asarray(frames, dtype=np.float32), allow_pickle=True, fix_imports=True)
    
    except FileNotFoundError:
        pass

    return np.asarray(frames, dtype=np.float32)
        
# Predict data

predictions = {n: 0 for n in saved_classes}
def predict_single_video(X):
    # Divide the total to get final probability
    total = 0
    for frame in X:
        new_frame = tf.expand_dims(frame,0)
        print(new_frame.shape)
        preds = model.predict(new_frame)
        # pred_value = np.argmax(preds)
        for i in range(len(preds)):
            print(preds[i])
            predictions[saved_classes[i]] += preds[i]
            total += preds[i]
        print(preds)
    for (key, val) in predictions.items():
        predictions[key] = val / total
    # final_prediction = max(predictions, key=predictions.get)
    return predictions


In [5]:
cam = cv2.VideoCapture(0)

i=0
frames=[]
while True:
    check, frame = cam.read()
    # print(frame)

    #inputting the image into the frames list.
    img=frame

    # #transform into numpy array
    w, h, c = img.shape

    if w < 226 or h < 226:
        d = 226. - min(w, h)
        sc = 1 + d / min(w, h)
        img = cv2.resize(img, dsize=(0, 0), fx=sc, fy=sc)

    if w > 256 or h > 256:
        img = cv2.resize(img, (math.ceil(w * (256 / w)), math.ceil(h * (256 / h))))

    img = (img / 255.) * 2 - 1

    frames.append(img)

    if i%25==0:
        get_files(frames)
    
    i+=1
    cv2.imshow('video', frame)


    key = cv2.waitKey(1)
    #breaks if we write escape, escape is key 27
    if key == 27:
        break
    # breaK

cam.release()
cv2.destroyAllWindows()

time.sleep(0.5)

# Process data

X = []
file_dir = "/Users/aly/Documents/Programming/Apps/Machine Learning/ASL Converter/training_models/demo_videos/demo_video_file.npy"
arr = np.load(file_dir)
for frame in arr:
    if len(X) == 0:
        X = np.array([frame])
    else:
        X = np.append(X, np.array([frame]), axis=0)
print(X.shape)

print(predict_single_video(X))

[[[-1.         -0.96078431 -0.95294118]
  [-0.97647059 -0.94509804 -0.9372549 ]
  [-1.         -0.99215686 -0.97647059]
  ...
  [ 0.27843137 -0.96078431 -0.35686275]
  [-0.52941176 -1.         -0.78039216]
  [-0.64705882 -1.         -0.94509804]]

 [[-1.         -0.96862745 -0.96078431]
  [-0.98431373 -0.95294118 -0.94509804]
  [-0.99215686 -0.96862745 -0.95294118]
  ...
  [ 0.50588235 -0.91372549 -0.20784314]
  [-0.49019608 -1.         -0.74117647]
  [-0.65490196 -1.         -0.95294118]]

 [[-1.         -0.97647059 -0.94509804]
  [-1.         -0.97647059 -0.96862745]
  [-1.         -0.96862745 -0.96078431]
  ...
  [ 0.27843137 -1.         -0.41960784]
  [-0.44313725 -0.99215686 -0.73333333]
  [-0.64705882 -1.         -0.9372549 ]]

 ...

 [[-1.         -0.99215686 -0.97647059]
  [-1.         -0.98431373 -0.96862745]
  [-1.         -0.99215686 -0.97647059]
  ...
  [-1.         -1.         -1.        ]
  [-0.99215686 -0.99215686 -0.99215686]
  [-1.         -1.         -1.        ]]

 [

2023-02-15 22:28:35.620814: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-15 22:28:35.745964: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 569ms/step
[9.9971849e-01 2.8067385e-04 8.2663979e-07 3.2446856e-08]
[[9.9971849e-01 2.8067385e-04 8.2663979e-07 3.2446856e-08]]
(1, 256, 256, 3)
1/1 [==============================] - 0s 24ms/step
[2.9415140e-01 7.0584798e-01 6.0990976e-07 1.7154176e-11]
[[2.9415140e-01 7.0584798e-01 6.0990976e-07 1.7154176e-11]]
(1, 256, 256, 3)
1/1 [==============================] - 0s 22ms/step
[4.4663990e-01 5.5335820e-01 1.9312172e-06 8.0297442e-11]
[[4.4663990e-01 5.5335820e-01 1.9312172e-06 8.0297442e-11]]
(1, 256, 256, 3)
1/1 [==============================] - 0s 21ms/step
[3.6131062e-02 9.6385920e-01 9.8102082e-06 1.3090762e-10]
[[3.6131062e-02 9.6385920e-01 9.8102082e-06 1.3090762e-10]]
(1, 256, 256, 3)
1/1 [==============================] - 0s 21ms/step
[3.8943467e-01 6.1055881e-01 6.4789538e-06 2.2147044e-09]
[[3.8943467e-01 6.1055881e-01 6.4789538e-06 2.2147044e-09]]
(1, 256, 256, 3)
1/1 [==============================] - 0s 21ms/step
[4.07881111e

: 

: 